# A BERT-based algorithm for edit distance between text trees

This notebook contains the implementation of a simple yet informative metric for text tree comparison. This way of text tree similarity measurement can be used, for example, to compare salient sentence-based mind maps generated by a neural network with reference maps. The way this algorithm works is by applying the Zhang-Shasha algorithm for tree edit distance to text trees, using semantic similarity as the cost of node updates. To measure semantic similarity of sentences in tree nodes, we use a BERT-like language model's embeddings of the sentences, given the context of all parent nodes if available, and compare these embeddings directly.

The Zhang-Shasha algorithm implementation is taken from the `zss` Python library developed by Tim Henderson and Steve Johnson (2013).

## Prerequisites

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer, SimilarityFunction

from tted.tree_format import Node, tree_with_context, json_to_node
from tted.computation import text_tree_distance, text_tree_distance_w_o_precomputation
from tted.baseline import baseline_distance


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\fedor\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\fedor\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\fedor\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\fedor\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
# Plot formatting
plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 12
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 24
plt.rcParams['axes.titlesize'] = 36
plt.rcParams['axes.labelsize'] = 24

We will use text trees stored in the JSON format as in the following example:
```json
{
  "A new algorithm for text tree edit distance based on Zhang-Shasha's algorithm and BERT-like model embedding similarity.": {
    "The algorithm's novelty is in its similarity measure based on BERT-like model embeddings.": {
      "Embedding distance is used as a measure of semantic similarity.": {},
      "The language model allows to capture semantic meaning of sentences and model their similarity.": {}
    },
    "Zhang-Shasha's algorithm is used to compute tree edit distance with new edit costs.": {
      "Semantic similarity is used as the update cost in the algorithm.": {},
      "The costs of insertion and removal of nodes are defined as the similarity of the node and an empty sentence.": {}
    },
    "The proposed algorithm is presented as a more informative metric of similarity between text trees.": {
      "The current ways of comparing text trees overlook their tree structure or the meaning of their labels.": {},
      "This new method can be used, for example, to compare mind maps or hierarchical summaries.": {}
    }
  }
}
```
We use the similarity metric from "Coherence Graph Guidance for Mind-Map Generation" (Zhang et al., 2024) as a baseline for comaprison. The full original code from this paper can be found at https://github.com/Cyno2232/CMGN.

## Basic tests

Below we provide an example use case of the functions above utilizing a model from `sentence_transformers`:

In [ ]:
A = (
    Node('We present a new metric for text tree comparison.', depth=0)
        .addkid(Node('It uses Zhang-Shasha\'s algorithm and a BERT-like model.', depth=1)
            .addkid(Node('Zhang-Shasha\'s algorithm is used to measure tree edit distance effectively.', depth=2))
            .addkid(Node('The BERT-like model is used to measure semantic similarity.', depth=2)))
        .addkid(Node('The algorithm is presented as an informative metric for text tree comparison.', depth=1)
            .addkid(Node('There hasn\'t yet been a metric that allows to compare tree-structured text data such as mind maps informatively.', depth=2))
            .addkid(Node('This metric can be used, for example, to evaluate automatic salient sentence-based mind map generation.', depth=2)))
)

B = (
    Node('A new metric for text tree comparison based on tree edit distance and semantic similarity.', depth=0)
        .addkid(Node('Zhang-Shasha\'s algorithm is used to compute tree edit distance.', depth=1))
        .addkid(Node('Semantic similarity is measured using a BERT-like language model.', depth=1)
            .addkid(Node('To measure it, the sentences with all parent nodes as context are passed to the language model.', depth=2))
            .addkid(Node('Semantic similarity is measured as the similarity of the model\'s embeddings of the sentences.', depth=2)))
        .addkid(Node('This metric can be used to compare text trees.', depth=1)
            .addkid(Node('For example, it can be utilized in automatic mind map generation evaluation against reference maps.', depth=2)))
)

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v1')
def cos_dist(a_embedding, b_embedding):
        return float(1 - model.similarity(a_embedding, b_embedding))

In [ ]:
%%time

AB_dist = text_tree_distance(A, B, model.encode, cos_dist)
print(AB_dist)

In [ ]:
%%time

AB_dist = text_tree_distance_w_o_precomputation(A, B, model.encode, cos_dist)
print(AB_dist)

In [ ]:
%%time

AB_base_dist = baseline_distance(A, B)
print(AB_base_dist)

## Main experiment

Now we can run a couple of experiments to evaluate these two metrics on simple test cases. We will compare the methods on three text tree sets, each based on the tree `C` from the example above:
1) A set of trees that are identical in semantic meaning and structure, but the sentences in the tree nodes are paraphrased;
2) A set of trees that are formed from the same sentences, but in different tree order;
3) A set of trees that are similar in structure and wording but significantly different in meaning.

For each set of trees we compute pairwise similarity scores with our metric and the baseline method. The goal is to capture the difference in meaning and structure of the trees while minimizing the distance between trees that are, in a sense, paraphrases of each other.

In [ ]:
def run_test(method, dist_args=None):
    scores = {}
    test_cases = ['paraphrase', 'meaning', 'restructure']

    total_time = 0
    for test_case in test_cases:
        path = 'data/' + test_case + '/' + test_case + '_'
        scores[test_case] = []

        trees = []
        for i in range(5):
            trees.append(json_to_node(path + str(i) + '.json'))

        start_time = time.time()
        for i in range(4):
            for j in range(i+1, 5):
                if method == 'tted':
                    scores[test_case].append(text_tree_distance(trees[i], trees[j], *dist_args))
                elif method == 'baseline':
                    scores[test_case].append(baseline_distance(trees[i], trees[j]))
                elif method == 'tted_w_o_precomputation':
                    scores[test_case].append(text_tree_distance_w_o_precomputation(trees[i], trees[j], *dist_args))
                else:
                    pass

        total_time += time.time() - start_time

    return scores, total_time

In [ ]:
def plot_scores(scores, plot_title, filename):
    fig, ax = plt.subplots(figsize=(7, 5))

    ax.boxplot(scores.values(), vert=False)

    ax.set_xlabel('Distance scores')
    ax.set_yticklabels(scores.keys())
    ax.grid()
    ax.set_title(plot_title)

    fig.savefig('../img/' + filename, bbox_inches='tight', pad_inches=0.1)

    plt.show()

In [ ]:
def result_frame(scores, method_name):
    results_array = [method_name]
    
    for exp in ('paraphrase', 'meaning', 'restructure'):
        results_array.append(np.mean(scores[exp]))
        results_array.append(np.std(scores[exp]))
    for exp in ('meaning', 'restructure'):
        results_array.append(np.mean(scores['paraphrase']) / np.mean(scores[exp]))
        results_array.append(np.sqrt((np.std(scores['paraphrase']) / np.mean(scores['paraphrase']))**2 +\
                                     (np.std(scores[exp]) / np.mean(scores[exp]))**2) * np.mean(scores['paraphrase']) / np.mean(scores[exp]))

    results_array = [results_array]
    frame = pd.DataFrame(results_array, columns=['Method', 
                                                 'Paraphrase mean', 
                                                 'Paraphrase std', 
                                                 'Meaning mean', 
                                                 'Meaning std', 
                                                 'Restructure mean',
                                                 'Restructure std',
                                                 'R_2 score',
                                                 'R_2 std',
                                                 'R_3 std',
                                                 'R_3 std'])
    return frame

Below are our experiments with several language models and distance metrics, with and without context usage, in comparison to the baseline method:

In [ ]:
scores_dict = {}
times_dict = {}

scores_dict['baseline'], times_dict['baseline'] = run_test('baseline')

for model_name in ['sentence-transformers/paraphrase-distilroberta-base-v1',
              'sentence-transformers/allenai-specter',
              'sentence-transformers/all-mpnet-base-v2',
              'sentence-transformers/paraphrase-multilingual-mpnet-base-v2']:
    model = SentenceTransformer(model_name)

    # The default similarity function for all of the models above is the cosine similarity function
    def dist(a_embedding, b_embedding):
        return float(1 - model.similarity(a_embedding, b_embedding))

    dist_args = [model.encode, dist, True]
    scores_dict[model_name], times_dict[model_name] = run_test('tted', dist_args)

# Experiment with different distance measures
for sim_measure, measure_name in zip([SimilarityFunction.EUCLIDEAN, SimilarityFunction.MANHATTAN], ['euclidian', 'manhattan']):
    model.similarity_fn_name = sim_measure
    def dist(embedding_a, embedding_b):
        return float(-model.similarity(embedding_a, embedding_b))
        
    scores_dict[measure_name], times_dict[measure_name] = run_test('tted', [model.encode, dist, True])

# Experiment without context usage
model.similarity_fn_name = SimilarityFunction.COSINE
def dist(a_embedding, b_embedding):
        return float(1 - model.similarity(a_embedding, b_embedding))

dist_args = [model.encode, dist, False]
scores_dict['Without context'], times_dict['Without context'] = run_test('tted', dist_args)

In [ ]:
times_dict

In [ ]:
frames_dict = {}
for exp_name, method_name in zip(scores_dict.keys(), ['Baseline method',
                                                    'TTED with paraphrase DistilRoBERTa',
                                                    'TTED with SPECTER',
                                                    'TTED with untuned MPNet',
                                                    'TTED with paraphrase MPNet',
                                                    'TTED with MPNet and Euclidian distance',
                                                    'TTED with MPNet and Manhattan distance',
                                                    'TTED with MPNet without context']):
    frames_dict[exp_name] = result_frame(scores_dict[exp_name], method_name)

In [ ]:
final_frame = pd.concat(frames_dict.values())
final_frame

In [ ]:
plot_scores(scores_dict['sentence-transformers/paraphrase-multilingual-mpnet-base-v2'], 
            'TTED with fine-tuned MPNet',
            '../paper/img/paraphrase_mpnet_results.png')
plot_scores(scores_dict['baseline'], 'Baseline method', '../paper/img/baseline_results.png')